In [2]:
from cobra import Model, Reaction, Metabolite
import tempfile
from pprint import pprint 
from cobra.io import write_sbml_model, validate_sbml_model
from cobra.io import load_model

### Model, Reactions, and Metabolites

In [3]:
model = Model('example_model') #creates new model object - model is the whole system 

reaction = Reaction('R_3OAS140') #creates new reaction object - reaction is, you guessed it, a single reaction 
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140' 
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0 #lowest rate at which the reaction can proceed, measured in mmol/gDW/h (millimoles per gram dry weight per hour)
reaction.upper_bound = 1000 
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x111b7d550
Stoichiometry,--> -->
GPR,
Lower bound,0
Upper bound,1000


In [4]:
model

Name,example_model
Memory address,1055a7a10
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [5]:
#creates metabolites - # reaction id's have to be Systems Biology Markup Language (SBML) compliant. 
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

In [6]:
reaction.add_metabolites({ #this is a dictionary of the metabolites and their stoichiometric coeffiecients. Notice how the metabolites are defined previously. 
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x111b7d550
Stoichiometry,ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c Dodecanoyl-ACP-n-C120ACP + H + Malonyl-acyl-carrier-protein --> acyl-carrier-protein + 3-Oxotetradecanoyl-acyl-carrier-protein + CO2
GPR,
Lower bound,0
Upper bound,1000


In [7]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )' #We saw this previously - this just adds genes involved in the reaction. 
reaction.genes

frozenset({<Gene STM1197 at 0x1121e4190>, <Gene STM2378 at 0x111b7f0e0>})

In [8]:
print(len(model.reactions), 'reactions') 
print(len(model.metabolites), 'metabolites') 
print(len(model.genes), 'genes') 

#This is just to show nothing we have not added all of the reaction definitions to the model. 


0 reactions
0 metabolites
0 genes


In [9]:
model.add_reactions([reaction]) #this line specifically adds the defined reaction to the model 

print(len(model.reactions), 'reactions') 
print(len(model.metabolites), 'metabolites') 
print(len(model.genes), 'genes') 

#This is to show that now everything we defined as been added to the model. 

1 reactions
6 metabolites
2 genes


In [10]:
print('Model Reactions')
print('--------------')
for x in model.reactions:
    print('%s : %s' % (x.id, x.reaction))

Model Reactions
--------------
R_3OAS140 : ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c


In [11]:
print('Metabolites')
print('-----------')
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

Metabolites
-----------
 malACP_c : C14H22N2O10PRS
      h_c : H
ddcaACP_c : C23H43N2O8PRS
    co2_c : CO2
    ACP_c : C11H21N2O7PRS
M3omrsACP_c : C25H45N2O9PRS


In [12]:
print('Genes')
print('-----------')
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print('%s is associated with reactions: %s' % (x.id, ', '.join(associated_ids)))

Genes
-----------
STM2378 is associated with reactions: R_3OAS140
STM1197 is associated with reactions: R_3OAS140


### Objective and Model Validation

In [13]:
model.objective = 'R_3OAS140' #sets the objective function of the model to be the reaction we just added.
print(model.objective.expression) #Algebraic expression of the model's objective function
print(model.objective.direction) #indicates the objective function is set to be maximized. 

1.0*R_3OAS140 - 1.0*R_3OAS140_reverse_60acb
max


In [14]:
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml: #creates and returns a temporary file
    write_sbml_model(model, filename=f_sbml.name) #writes the cobra model to the file name 
    report = validate_sbml_model(filename=f_sbml.name) #Validates the model and returns it along with a list of errors. 

pprint(report) #There are no errors in the model. 

(<Model example_model at 0x1122f0550>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


### Exchanges, Sinks, and Demands

In [15]:
# boudnary reactions represent the transport of metabolites between cells and their environment. There are three types; exchange, demand, and sink
print('exchanges', model.exchanges)
print('demands', model.demands)
print('sinks', model.sinks)

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


exchanges []
demands []
sinks []


In [16]:
model.add_metabolites([Metabolite('glycogen_c', name='glycogen', compartment='c'), Metabolite('co2_e', name='CO2', compartment='e')]) #  c = cytosol, e = extracellular 

model.metabolites

[<Metabolite malACP_c at 0x10574b230>,
 <Metabolite h_c at 0x111b3bce0>,
 <Metabolite ddcaACP_c at 0x111b3ccb0>,
 <Metabolite co2_c at 0x111b0fed0>,
 <Metabolite ACP_c at 0x111b7ee40>,
 <Metabolite M3omrsACP_c at 0x111b0f9d0>,
 <Metabolite glycogen_c at 0x112175020>,
 <Metabolite co2_e at 0x112309810>]

In [17]:
model.add_boundary(model.metabolites.get_by_id('co2_e'), type='exchange') #adds a boundary reaction for a specific type of metabolite 

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x1122f1450
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [18]:
model.add_boundary(model.metabolites.get_by_id('glycogen_c'), type = 'sink')

Reaction identifier,SK_glycogen_c
Name,glycogen sink
Memory address,0x111b3ac40
Stoichiometry,glycogen_c <=> glycogen <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [19]:
print('exchanges', model.exchanges) #These just show we've added an exchange and a sink reaction 
print('sinks', model.sinks)
print('demands', model.demands)

exchanges [<Reaction EX_co2_e at 0x1122f1450>]
sinks [<Reaction SK_glycogen_c at 0x111b3ac40>]
demands []


In [20]:
model.boundary #easier way to get all boundary reactions. 

[<Reaction EX_co2_e at 0x1122f1450>, <Reaction SK_glycogen_c at 0x111b3ac40>]

### Simulating with FBA

In [26]:
model = load_model("textbook")
solution = model.optimize() #
solution_faster = model.slim_optimize()
print(solution) #shows what the model predicts the biomass growth will be
print(solution.objective_value)
print(solution.status) 
print(solution.fluxes) #pandas series of reactions indexed by reaction identifier. 
print(solution.shadow_prices) #How much the objective value would improve if the availability of a particular resource were increased by one unit
print(model.objective)

<Solution 0.874 at 0x111b3afd0>
0.8739215069684295
optimal
ACALD     0.000000e+00
ACALDt    0.000000e+00
ACKr      1.885004e-15
ACONTa    6.007250e+00
ACONTb    6.007250e+00
              ...     
TALA      1.496984e+00
THD2      0.000000e+00
TKT1      1.496984e+00
TKT2      1.181498e+00
TPI       7.477382e+00
Name: fluxes, Length: 95, dtype: float64
13dpg_c     -0.047105
2pg_c       -0.042013
3pg_c       -0.042013
6pgc_c      -0.091665
6pgl_c      -0.090392
               ...   
s7p_c       -0.113308
succ_c      -0.050925
succ_e      -0.048379
succoa_c    -0.054744
xu5p__D_c   -0.082753
Name: shadow_prices, Length: 72, dtype: float64
Maximize
1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


In [31]:
model.summary() #This is super useful. 

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [ ]:
model.metabolites.nadh_c.summary() #producing reactions is what produces NADH. Consuming reactions is what consumes NADH, in this specific query

Percent,Flux,Reaction,Definition
13.14%,5.064,AKGDH,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
8.04%,3.1,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
41.58%,16.02,GAPD,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13.14%,5.064,MDH,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
24.09%,9.283,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-38.53,NADH16,4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q8h2_c


In [29]:
model.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
66.58%,45.51,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
23.44%,16.02,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
2.57%,1.758,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
7.41%,5.064,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
Percent,Flux,Reaction,Definition
12.27%,-8.39,ATPM,atp_c + h2o_c --> adp_c + h_c + pi_c
76.46%,-52.27,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
0.33%,-0.2235,GLNS,atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c
10.94%,-7.477,PFK,atp_c + f6p_c --> adp_c + fdp_c + h_c


In [ ]:
biomass_rxn = model.reactions.get_by_id('Biomass_Ecoli_core')
print(biomass_rxn) #This just gets the objective function reaction 

Biomass_Ecoli_core: 1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c


### This just resets the objective function to the ATP maintenance reaction, or the non-growth related ATP req's. 

In [ ]:
model.objective = "ATPM" #
atp_reaction = model.reactions.get_by_id('ATPM')
atp_reaction
model.reactions.get_by_id("ATPM").upper_bound = 1000
model.optimize().objective_value

175.0